In [ ]:
#  !pip install transformers seqeval[gpu]

In [1]:
!pip install transformers==3.1.0

     |████████████████████████████████| 884 kB 5.2 MB/s 
     |████████████████████████████████| 1.2 MB 40.0 MB/s 
     |████████████████████████████████| 3.0 MB 38.1 MB/s 
     |████████████████████████████████| 880 kB 35.1 MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895260 sha256=2557791b251217c8c1e7f73a698bf2229a7b1b8fb83007df51d04c184c593e3e
  Stored in directory: /root/.cache/pip/wheels/87/39/dd/a83eeef36d0bf98e7a4d1933a4ad2d660295a40613079bafc9
Successfully built sacremoses


In [2]:
from pprint import pprint
from google.colab import drive
from google.colab import files
import regex as re
import pandas as pd
import numpy as np
from transformers import BertTokenizerFast, BertConfig, BertForTokenClassification
import torch
from torch.utils.data import Dataset, DataLoader
from torch import cuda
from sklearn.metrics import accuracy_score

import glob
import os.path
import numpy as np
import sys
import codecs

In [3]:
device = 'cuda' if cuda.is_available() else 'cpu'
print(device)

cpu


In [4]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [5]:
train_folder = "/content/drive/MyDrive/NLP/project_5_data/datasets/train-articles" 
dev_folder = "/content/drive/MyDrive/NLP/project_5_data/datasets/dev-articles"    
train_labels_file = "/content/drive/MyDrive/NLP/project_5_data/datasets/train-labels-task-si/"
dev_labels_file = "/content/drive/MyDrive/NLP/project_5_data/datasets/dev-labels-task-si"
task_TC_output_file = "/content/drive/MyDrive/NLP/project_5_data/baseline-output-TC.txt"

In [6]:

def read_articles_from_file_list(folder_name, file_pattern="*.txt"):
    """
    Read articles from files matching patterns <file_pattern> from  
    the directory <folder_name>. 
    The content of the article is saved in the dictionary whose key
    is the id of the article (extracted from the file name).
    Each element of <sentence_list> is one line of the article.
    """
    file_list = glob.glob(os.path.join(folder_name, file_pattern))
    print(folder_name, len(file_list))
    articles = {}
    article_id_list, sentence_id_list, sentence_list = ([], [], [])
    for filename in sorted(file_list):
        article_id = os.path.basename(filename).split(".")[0][7:]
        with codecs.open(filename, "r", encoding="utf8") as f:
          # sentence = ' '.join([line.strip() for line in f if len(line) >= 1])
          # articles[article_id] = sentence
          sentence = f.read().strip()
          sentence = sentence.replace('\n\n', ' ').strip().replace('\n', ' ').strip()
          articles[article_id] = sentence
    return articles


def read_labels_from_file_list(folder_name, file_pattern="*.txt"):

    file_list = glob.glob(os.path.join(folder_name, file_pattern))
    print(folder_name, len(file_list))
    labels = {}
    for filename in sorted(file_list):
        article_id = os.path.basename(filename).split(".")[0][7:]
        if article_id not in labels:
          labels[article_id] = []
        with codecs.open(filename, "r", encoding="utf8") as f:
          for line in f:
            line_values = line.strip().split('\t')
            article_id = line_values[0]
            start = line_values[1]
            end = line_values[2]
            labels[article_id].append((article_id, start, end))
    return labels

In [ ]:
a = read_labels_from_file_list('/content/drive/MyDrive/NLP/project_5_data/datasets/dev-labels-task-si','*.labels')

/content/drive/MyDrive/NLP/project_5_data/datasets/dev-labels-task-si 75


In [ ]:
a['730246508']

[('730246508', '1654', '1676'),
 ('730246508', '4557', '4573'),
 ('730246508', '4406', '4412'),
 ('730246508', '3840', '3948'),
 ('730246508', '4187', '4195'),
 ('730246508', '5052', '5098')]

In [7]:
dev_folder = "/content/drive/MyDrive/NLP/project_5_data/datasets/dev-articles" # check that the path to the datasets folder is correct, if not adjust these variables accordingly 
propaganda_techniques_file = "/content/drive/MyDrive/NLP/project_5_data/propaganda-techniques-scorer/data/propaganda-techniques-names-semeval2020task11.txt" # propaganda_techniques_file is in the tools.tgz file (download it from the team page)
task_SI_output_file = "/content/drive/MyDrive/NLP/project_5_data/baseline-output-SI.txt"

In [ ]:
# !python3 task-SI_scorer.py -s project_5_data/baseline-output-SI.txt -r project_5_data/datasets/dev-labels-task-si/

In [8]:
file_list = glob.glob(os.path.join(train_folder, "*.txt"))
train_articles_content, train_articles_id = ([], [])
for filename in file_list:
    with open(filename, "r", encoding="utf-8") as f:
        train_articles_content.append(' '.join([line.strip() for line in f]))
        train_articles_id.append(os.path.basename(filename).split(".")[0][7:])


In [9]:
train_articles = read_articles_from_file_list(train_folder)
dev_articles = read_articles_from_file_list(dev_folder)

/content/drive/MyDrive/NLP/project_5_data/datasets/train-articles 371
/content/drive/MyDrive/NLP/project_5_data/datasets/dev-articles 75


In [10]:
train_articles['111111121']
dev_articles['730081389']

'Police had previously gone to home where Ohio patrol officers were killed CLEVELAND — Police invstigating domestic disputes had previously gone to the home where a man fatally shot two police officers over the weekend, but no arrests were ever made, police reports from the Columbus suburb of Westerville show. Westerville Officers Eric Joering, 39, and Anthony Morelli, 54, were killed shortly after noon Saturday in this normally quiet suburb while responding to a 911 hang-up call. The suspect, 30-year-old Quentin Smith, was shot and wounded by the officers and taken to Ohio State University Wexner Medical Center in critical condition Saturday. Advertisement A series of 911 calls released by the city of Westerville provide some details about what happened Saturday at a complex of town houses. Smith lived there with his wife, Candace, and a young daughter. Get Ground Game in your inbox: Daily updates and analysis on national politics from James Pindell. Sign Up Thank you for signing up! 

In [11]:
print(len(train_articles))
print(len(dev_articles))

371
75


In [12]:
TASK_3_ARTICLE_ID_COL=0
#TASK_3_TECHNIQUE_NAME_COL=1
TASK_3_FRAGMENT_START_COL=1
TASK_3_FRAGMENT_END_COL=2
def extract_article_id_from_file_name(fullpathfilename):

    regex = re.compile("article([0-9]+).*")
    return regex.match(os.path.basename(fullpathfilename)).group(1)

   
def load_annotation_list_from_folder(folder_name, techniques_names):

    file_list = glob.glob(os.path.join(folder_name, "*.labels"))
    if len(file_list)==0:
        print("Cannot load file list in folder " + folder_name)
        sys.exit()
    annotations = {}
    for filename in file_list:
        annotations[extract_article_id_from_file_name(filename)] = []
        with open(filename, "r") as f:
            for row_number, line in enumerate(f.readlines()):
                row = line.rstrip().split("\t")
                annotations[row[TASK_3_ARTICLE_ID_COL]].append((row[TASK_3_FRAGMENT_START_COL],row[TASK_3_FRAGMENT_END_COL]))
    return annotations

In [13]:
techniques_names = [ "propaganda" ]
train_annotation = load_annotation_list_from_folder(train_labels_file, techniques_names)
new_annotations = read_labels_from_file_list(train_labels_file,'*.labels')

/content/drive/MyDrive/NLP/project_5_data/datasets/train-labels-task-si/ 371


In [14]:
train_annotation.keys(), new_annotations.keys()

(dict_keys(['736757214', '790720480', '754179642', '729651527', '712382330', '787668628', '721710340', '715588833', '755814432', '727128296', '111111124', '782086447', '758512204', '761780613', '778139122', '111111112', '790777028', '732708002', '789121798', '698092698', '727569885', '757713354', '793467906', '704591553', '697444415', '723501118', '770376380', '725276027', '758882558', '694356862', '757243988', '774007496', '696246189', '762200179', '789370998', '729549972', '728972961', '706088110', '731063195', '701447437', '710376094', '999000870', '729303442', '715790840', '721890296', '787955075', '725498022', '707451080', '781768042', '999001296', '999000155', '764664283', '763260610', '727736595', '999000159', '795693029', '999000133', '752287274', '758472954', '721406153', '795079843', '730389374', '727736557', '773748383', '700461600', '708071969', '766942310', '707772906', '769962236', '729668796', '758386255', '776345502', '697454736', '702077434', '788816478', '728758697', 

In [15]:
train_labels = {}
for article in train_articles.keys():
    labels = [0] * len(train_articles[article])
    for annot in train_annotation[article]:
        labels[int(annot[0]) : int(annot[1])] = [1] * (int(annot[1]) - int(annot[0]))
    train_labels[article] = labels

In [16]:
train_labels['111111111']
len(train_articles['111111111'].split(' '))

371

In [17]:
train_labels_str = {}
for article in train_articles.keys():
    labels = [0] * len(train_articles[article].split(' '))

    index = 0
    
    word_index = -1
    for word in train_articles[article].split(' '):
      word_index+=1
      if(train_labels[article][index] == 1):
        labels[word_index] = 1
      index+= len(word) + 1
    
    str_labels = []
    span_started = False
    for label in labels:
      if label == 0:
        str_labels.append('O')
        span_started = False
      elif label == 1 and not span_started:
        str_labels.append('I-Prop')
        span_started = True
      elif label == 1 and span_started:
        str_labels.append('I-Prop')
        span_started = True
    train_labels[article] = labels
    train_labels_str[article] = str_labels

In [18]:
def get_train_labels(data_articles, data_annotation):
    
  train_labels = {}
  for article in data_articles.keys():
      labels = [0] * len(data_articles[article])
      for annot in data_annotation[article]:
          labels[int(annot[0]) : int(annot[1])] = [1] * (int(annot[1]) - int(annot[0]))
      train_labels[article] = labels

  train_labels_str = {}
  for article in data_articles.keys():
      labels = [0] * len(data_articles[article].split(' '))

      index = 0
      
      word_index = -1
      for word in data_articles[article].split(' '):
        word_index+=1
        if(train_labels[article][index] == 1):
          labels[word_index] = 1
        index+= len(word) + 1
      
      str_labels = []
      span_started = False
      for label in labels:
        if label == 0:
          str_labels.append('O')
          span_started = False
        elif label == 1 and not span_started:
          str_labels.append('I-Prop')
          span_started = True
        elif label == 1 and span_started:
          str_labels.append('I-Prop')
          span_started = True
      train_labels[article] = labels
      train_labels_str[article] = str_labels

  return train_labels, train_labels_str

In [19]:
len(train_labels), len(train_labels_str)

(371, 371)

In [20]:
sum(train_labels['111111111'])

76

In [21]:
train_annotation['111111111'], train_articles['111111111']

([('265', '323'),
  ('1795', '1935'),
  ('149', '157'),
  ('1069', '1091'),
  ('1334', '1462'),
  ('1577', '1616'),
  ('2023', '2086')],
 'Next plague outbreak in Madagascar could be \'stronger\': WHO Geneva - The World Health Organisation chief on Wednesday said a deadly plague epidemic appeared to have been brought under control in Madagascar, but warned the next outbreak would likely be stronger. "The next transmission could be more pronounced or stronger," WHO Director-General Tedros Adhanom Ghebreyesus told reporters in Geneva, insisting that "the issue is serious." An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August. Madagascar has suffered bubonic plague outbreaks almost every year since 1980, often caused by rats fleeing forest fires. The disease tends to make a comeback each hot rainy season, from September to April. On 

In [22]:
train_df = pd.DataFrame(columns = ['ID','text','labels'])
count = 0
for article in train_articles.keys():
    train_df.loc[count,'ID'] = article
    train_df.loc[count,'text'] = train_articles[article]
    temp_label = [label for label in train_labels_str[article]]
    train_df.loc[count,'labels'] = ','.join(temp_label)
    count+=1
del train_df['ID']

In [23]:
train_df

,text,labels
0,Next plague outbreak in Madagascar could be 's...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
1,US bloggers banned from entering UK Two promin...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
2,Kate Steinle's death at the hands of a Mexican...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
3,U.S. judge frees Indonesian immigrant held by ...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
4,Here are all the sexual misconduct accusations...,"O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,..."
...,...,...
366,Altered Election Documents Tied To Florida Dem...,"O,O,O,O,O,O,O,O,O,O,O,O,I-Prop,I-Prop,I-Prop,O..."
367,Migrant Caravan Reach Border & Climb Atop Fenc...,"O,O,O,O,O,I-Prop,I-Prop,I-Prop,I-Prop,O,O,O,O,..."
368,Guardian ups its vilification of Julian Assang...,"O,O,O,I-Prop,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O..."
369,This Guardian Fake News Story Proves That The ...,"I-Prop,I-Prop,I-Prop,I-Prop,I-Prop,I-Prop,I-Pr..."


In [ ]:
# len(train_df.loc[20,'text'].split(' '))

In [24]:
print(len(train_df.text[0].strip().split(' ')))
train_df.text[0], train_df.labels[0]

371


('Next plague outbreak in Madagascar could be \'stronger\': WHO Geneva - The World Health Organisation chief on Wednesday said a deadly plague epidemic appeared to have been brought under control in Madagascar, but warned the next outbreak would likely be stronger. "The next transmission could be more pronounced or stronger," WHO Director-General Tedros Adhanom Ghebreyesus told reporters in Geneva, insisting that "the issue is serious." An outbreak of both bubonic plague, which is spread by infected rats via flea bites, and pneumonic plague, spread person to person, has killed more than 200 people in the Indian Ocean island nation since August. Madagascar has suffered bubonic plague outbreaks almost every year since 1980, often caused by rats fleeing forest fires. The disease tends to make a comeback each hot rainy season, from September to April. On average, between 300 and 600 infections are recorded every year among a population approaching 25 million people, according to a UN estim

In [25]:
len(train_df.labels[0].split(','))

371

In [ ]:
# np.mean([len(train_df.loc[i,'text'].split(' ')) for i in range(len(train_df))])

In [26]:
# labels_to_ids = {k: v for v, k in enumerate(['B-Prop','I-Prop', 'O'])}
# ids_to_labels = {v: k for v, k in enumerate(['B-Prop','I-Prop', 'O'])}
# labels_to_ids, ids_to_labels

labels_to_ids = {k: v for v, k in enumerate(['I-Prop', 'O'])}
ids_to_labels = {v: k for v, k in enumerate(['I-Prop', 'O'])}
labels_to_ids, ids_to_labels

({'I-Prop': 0, 'O': 1}, {0: 'I-Prop', 1: 'O'})

In [27]:
MAX_LEN = 512
TRAIN_BATCH_SIZE = 2
VALID_BATCH_SIZE = 2
DEV_BATCH_SIZE = 2
EPOCHS = 5
LEARNING_RATE = 1e-05
MAX_GRAD_NORM = 10
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

In [29]:
class dataset(Dataset):
  def __init__(self, dataframe, tokenizer, max_len):
        self.len = len(dataframe)
        self.data = dataframe
        self.tokenizer = tokenizer
        self.max_len = max_len

  def __getitem__(self, index):
        # step 1: get the sentence and word labels 
        sentence = self.data.text[index].strip().split(' ')  
        word_labels = self.data.labels[index].split(",") 


        # step 2: use tokenizer to encode sentence (includes padding/truncation up to max length)
        encoding = self.tokenizer(sentence,
                             is_pretokenized=True, 
                             return_offsets_mapping=True, 
                             padding='max_length', 
                             truncation=True, 
                             max_length=self.max_len)

        # step 3: create token labels only for first word pieces of each tokenized word
        labels = [labels_to_ids[label] for label in word_labels] 

        # create an empty array of -100 of length max_length
        encoded_labels = np.ones(len(encoding["offset_mapping"]), dtype=int) * -100
        i = 0
        for idx, mapping in enumerate(encoding["offset_mapping"]):
          if mapping[0] == 0 and mapping[1] != 0:
            # overwrite label
            encoded_labels[idx] = labels[i]
            i += 1

        # step 4: turn everything into PyTorch tensors
        item = {key: torch.as_tensor(val) for key, val in encoding.items()}
        item['labels'] = torch.as_tensor(encoded_labels)
        
        return item

  def __len__(self):
        return self.len

In [30]:
train_size = 1
train_dataset = train_df.sample(frac=train_size,random_state=200)
test_dataset = train_df.drop(train_dataset.index).reset_index(drop=True)
train_dataset = train_dataset.reset_index(drop=True)

print("FULL Dataset: {}".format(train_df.shape))
print("TRAIN Dataset: {}".format(train_dataset.shape))
print("TEST Dataset: {}".format(test_dataset.shape))

training_set = dataset(train_dataset, tokenizer, MAX_LEN)
testing_set = dataset(test_dataset, tokenizer, MAX_LEN)

FULL Dataset: (371, 2)
TRAIN Dataset: (371, 2)
TEST Dataset: (0, 2)


In [31]:
(training_set[1])

{'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1

In [ ]:
for token, mapping, label in zip(tokenizer.convert_ids_to_tokens(training_set[1]["input_ids"]), training_set[1]['offset_mapping'], training_set[1]["labels"]):
  print('{0:10} {1}'.format(token, label), '  ', mapping)

In [32]:
train_params = {'batch_size': TRAIN_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

test_params = {'batch_size': VALID_BATCH_SIZE,
                'shuffle': True,
                'num_workers': 0
                }

training_loader = DataLoader(training_set, **train_params)
# testing_loader = DataLoader(testing_set, **test_params)

In [33]:
model = BertForTokenClassification.from_pretrained('bert-base-uncased', num_labels=len(labels_to_ids))
model.to(device)

Downloading:   0%|          | 0.00/433 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-u

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwis

In [34]:
inputs = training_set[2]
input_ids = inputs["input_ids"].unsqueeze(0)
attention_mask = inputs["attention_mask"].unsqueeze(0)
labels = inputs["labels"].unsqueeze(0)

input_ids = input_ids.to(device)
attention_mask = attention_mask.to(device)
labels = labels.to(device)

outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
initial_loss = outputs[0]
initial_loss

tensor(0.6812, grad_fn=<NllLossBackward0>)

In [35]:
input_ids.shape

torch.Size([1, 512])

In [36]:
labels.shape

torch.Size([1, 512])

In [37]:
tr_logits = outputs[1]
tr_logits.shape

torch.Size([1, 512, 2])

In [38]:
optimizer = torch.optim.Adam(params=model.parameters(), lr=LEARNING_RATE)

In [39]:
# Defining the training function on the 80% of the dataset for tuning the bert model
def train(epoch):
    tr_loss, tr_accuracy = 0, 0
    nb_tr_examples, nb_tr_steps = 0, 0
    tr_preds, tr_labels = [], []
    # put model in training mode
    model.train()
    
    for idx, batch in enumerate(training_loader):
        
        ids = batch['input_ids'].to(device, dtype = torch.long)
        mask = batch['attention_mask'].to(device, dtype = torch.long)
        labels = batch['labels'].to(device, dtype = torch.long)

        loss, tr_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
        tr_loss += loss.item()

        nb_tr_steps += 1
        nb_tr_examples += labels.size(0)
        
        if idx % 100==0:
            loss_step = tr_loss/nb_tr_steps
            print(f"Training loss per 100 training steps: {loss_step}")
           
        # compute training accuracy
        flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
        active_logits = tr_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
        flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
        
        # only compute accuracy at active labels
        active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
        #active_labels = torch.where(active_accuracy, labels.view(-1), torch.tensor(-100).type_as(labels))
        
        labels = torch.masked_select(flattened_targets, active_accuracy)
        predictions = torch.masked_select(flattened_predictions, active_accuracy)
        
        tr_labels.extend(labels)
        tr_preds.extend(predictions)

        tmp_tr_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
        tr_accuracy += tmp_tr_accuracy
    
        # gradient clipping
        torch.nn.utils.clip_grad_norm_(
            parameters=model.parameters(), max_norm=MAX_GRAD_NORM
        )
        
        # backward pass
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    epoch_loss = tr_loss / nb_tr_steps
    tr_accuracy = tr_accuracy / nb_tr_steps
    print(f"Training loss epoch: {epoch_loss}")
    print(f"Training accuracy epoch: {tr_accuracy}")

In [40]:
torch.cuda.empty_cache()

In [ ]:
for epoch in range(EPOCHS):
    print(f"Training epoch: {epoch + 1}")
    train(epoch)

In [42]:
ids_to_labels[-100] = 'O'

In [43]:
def valid(model, testing_loader):
    # put model in evaluation mode
    model.eval()
    
    eval_loss, eval_accuracy = 0, 0
    nb_eval_examples, nb_eval_steps = 0, 0
    eval_preds, eval_labels = [], []
    
    with torch.no_grad():
        for idx, batch in enumerate(testing_loader):
            
            ids = batch['input_ids'].to(device, dtype = torch.long)
            mask = batch['attention_mask'].to(device, dtype = torch.long)
            labels = batch['labels'].to(device, dtype = torch.long)
            # temp_labels = labels
            # print(idx,labels)
            loss, eval_logits = model(input_ids=ids, attention_mask=mask, labels=labels)
            
            eval_loss += loss.item()

            nb_eval_steps += 1
            nb_eval_examples += labels.size(0)
        
            if idx % 100==0:
                loss_step = eval_loss/nb_eval_steps
                print(f"Validation loss per 100 evaluation steps: {loss_step}")
              
            # compute evaluation accuracy
            flattened_targets = labels.view(-1) # shape (batch_size * seq_len,)
            active_logits = eval_logits.view(-1, model.num_labels) # shape (batch_size * seq_len, num_labels)
            flattened_predictions = torch.argmax(active_logits, axis=1) # shape (batch_size * seq_len,)
            
            # only compute accuracy at active labels
            active_accuracy = labels.view(-1) != -100 # shape (batch_size, seq_len)
            labels = torch.masked_select(flattened_targets, active_accuracy)
            predictions = torch.masked_select(flattened_predictions, active_accuracy)
            # print(temp_labels == labels)

            eval_labels.extend(labels)
            eval_preds.extend(predictions)
            # eval_labels.extend(flattened_targets)
            # eval_preds.extend(flattened_predictions)
            
            # tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            tmp_eval_accuracy = accuracy_score(labels.cpu().numpy(), predictions.cpu().numpy())
            eval_accuracy += tmp_eval_accuracy

    labels = [ids_to_labels[id.item()] for id in eval_labels]
    predictions = [ids_to_labels[id.item()] for id in eval_preds]
    
    eval_loss = eval_loss / nb_eval_steps
    eval_accuracy = eval_accuracy / nb_eval_steps
    print(f"Validation Loss: {eval_loss}")
    print(f"Validation Accuracy: {eval_accuracy}")

    return labels, predictions

In [ ]:
dev_annotation = load_annotation_list_from_folder(dev_labels_file, techniques_names)
dev_labels = {}
for article in dev_articles.keys():
    labels = [0] * len(dev_articles[article])
    for annot in dev_annotation[article]:
        labels[int(annot[0]):int(annot[1])] = [1] * (int(annot[1]) - int(annot[0]))
    dev_labels[article] = labels

dev_labels_str = {}
for article in dev_articles.keys():
    labels = [0] * len(dev_articles[article].split(' '))

    index = 0
    
    word_index = -1
    for word in dev_articles[article].split(' '):
      word_index+=1
      if (index<len(dev_labels[article])) and (dev_labels[article][index] == 1):
        labels[word_index] = 1
      index+= len(word) + 1

    str_labels = []
    span_started = False
    for label in labels:
      if label == 0:
        str_labels.append('O')
        span_started = False
      elif label == 1 and not span_started:
        str_labels.append('I-Prop')
        span_started = True
      elif label == 1 and span_started:
        str_labels.append('I-Prop')
        span_started = True

    dev_labels[article] = labels
    dev_labels_str[article] = str_labels

dev_df = pd.DataFrame(columns = ['ID','text','labels'])
count = 0
for article in dev_articles.keys():
    dev_df.loc[count,'ID'] = article
    dev_df.loc[count,'text'] = dev_articles[article]
    # temp_label = [str(int) for int in dev_labels[article]]
    temp_label = [label for label in dev_labels_str[article]]
    dev_df.loc[count,'labels'] = ','.join(temp_label)
    count+=1

dev_id = dev_df['ID']
del dev_df['ID']

In [ ]:
# dev_df.loc[1,'text']

In [ ]:
# dev_df.loc[1,'text'][1370:1393]

In [ ]:
# dev_df.loc[1,'text'].split(' ').index('American')

In [ ]:
# dev_df.loc[1,'labels'].split(',')[222]

In [ ]:
print(len(dev_df.loc[1,'text'].split(' ')))
print(len(dev_df.loc[1,'labels'].split(",")))
dev_dataset = pd.DataFrame(dev_df.loc[0])
data_updated = pd.DataFrame(columns = ['text','labels'])
data_updated.loc[0, 'text'] = dev_df.loc[1].text
data_updated.loc[0, 'labels'] = dev_df.loc[1].labels

data_updated
[i for i in range(5)]

In [ ]:
#All data in dev set
dev_dataset = dev_df

dev_set = dataset(dev_dataset, tokenizer, MAX_LEN)
dev_params = {'batch_size': DEV_BATCH_SIZE,
                'shuffle': False,
                'num_workers': 0
                }


dev_loader = DataLoader(dev_set, **dev_params)

In [ ]:
# Run 1 by 1
prediction_map = {}
actual_labels_map = {}

for i in range(len(dev_df)):
    single_dev_row = pd.DataFrame(columns = ['text','labels'])
    single_dev_row.loc[0, 'text'] = dev_df.loc[i].text
    single_dev_row.loc[0, 'labels'] = dev_df.loc[i].labels

    dev_set = dataset(single_dev_row, tokenizer, MAX_LEN)
    dev_loader = DataLoader(dev_set, **dev_params)
    labels, predictions = valid(model, dev_loader)
    prediction_map[dev_id[i]] = predictions
    actual_labels_map[dev_id[i]] = labels

In [ ]:
labels_actual, predictions = valid(model, dev_loader)

In [ ]:
total_word_len = []
for article in dev_df.text:
    total_word_len.append(len(article.split(' ')))
np.mean(total_word_len), len(dev_df.text)


In [ ]:
predictions.count('I-Prop'), predictions.count('B-Prop'), predictions.count('O'), len(predictions)

In [ ]:
correct_word_prediction = 0
for index in range(len(labels)):
  if labels[index] == predictions[index] and labels[index] is not 'O':
    correct_word_prediction+=1
    print(index, labels[index])

print(correct_word_prediction)

In [ ]:
task_SI_output_file = '/content/drive/MyDrive/NLP/project_5_data/baseline-output-SI.txt'

In [ ]:
with open(task_SI_output_file, "w") as fout:

    for dev_article_id in dev_id:
      article = dev_articles[dev_article_id]
      predictions = prediction_map[dev_article_id]
      # print("predictions: ", predictions)
      labels = [0] * len(predictions)
      for i in range(len(predictions)):
        if predictions[i] == 'B-Prop' or predictions[i] == 'I-Prop':
          labels[i] = 1
        else:
          labels[i] = 0
      
    #   print("Labels: ", labels)
    #   if 1 in labels:
    #     print(labels.count(1))
    #   print("Article: ", article)
    #   print(len(article))

      char_index = 0
      span_started = False
      span_len = 0
      start = 0
      label_index = 0
      for idx in range(len(article)):
        if article[idx] == ' ' and span_started:
          label_index += 1
          span_len += 1
          continue
        elif article[idx] == ' ':
          label_index += 1
          continue
        if label_index >= len(labels):
          break
        if labels[label_index] == 1 and not span_started:
          span_started = True          
          start = idx
          span_len += 1 
        elif labels[label_index] == 1 and span_started:        
          span_len += 1           
        elif labels[label_index] == 0 and span_started:
          span_started = False          
          if span_len > 0:
            fout.write("%s\t%s\t%s\n" % (dev_article_id, start, start+span_len))
            print(dev_article_id, start, start+span_len)
            span_len = 0

      print(label_index)
            

288
730093263 1375 1384
730093263 2062 2067
730093263 2072 2102
396
730246508 1659 1676
730246508 1693 1702
730246508 1705 1711
730246508 1775 1779
730246508 1785 1794
730246508 1810 1818
385
140
738028498 0 33
738028498 165 169
738028498 174 177
738028498 186 198
738028498 201 207
738028498 323 328
738028498 344 350
738028498 363 370
738028498 587 590
738028498 613 637
738028498 695 739
738028498 962 969
738028498 995 1002
738028498 1014 1052
738028498 1060 1120
738028498 1126 1129
738028498 1133 1255
738028498 1399 1468
738028498 1504 1535
738028498 1537 1553
738028498 1567 1592
738028498 1922 1949
738028498 1968 1986
738028498 2267 2281
392
738361208 991 1039
738361208 1090 1095
374
738442776 1330 1368
738442776 1690 1805
738442776 1856 1878
738442776 2113 2133
738442776 2149 2206
738442776 2228 2236
738442776 2268 2272
371
738447109 420 509
738447109 1904 1919
738447109 1927 1943
405
320
389
740235127 477 484
740235127 703 716
740235127 752 758
740235127 768 783
740235127 816 821
7

In [ ]:
%cd /content/drive/MyDrive/NLP/
!python3 task-SI_scorer.py -s project_5_data/baseline-output-SI.txt -r project_5_data/datasets/dev-labels-task-si/

/content/drive/MyDrive/NLP
2022-03-27 05:09:11,884 - INFO - Checking user submitted file
2022-03-27 05:09:12,004 - INFO - Scoring the submission with precision and recall method
2022-03-27 05:09:12,081 - INFO - Precision=145.427868/355=0.409656	Recall=136.975633/940=0.145719
2022-03-27 05:09:12,081 - INFO - F1=0.214970
